In [18]:
import gensim
import os
import collections
import smart_open
import random
import pandas as pd

In [103]:
# Set file names for train and test data
#test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
#lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
#lee_test_file = test_data_dir + os.sep + 'lee.cor'
filepath_train = "/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/featureExtraction/EnglishCleanedTrainingData (1).csv"
train_data = pd.read_csv(filepath_train)
filepath_test = "/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/featureExtraction/EnglishCleanedTestingData (1).csv"
test_data = pd.read_csv(filepath_test)

In [113]:
def read_corpus(fname, data, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i in range(0, len(data)):
            if tokens_only:
                yield gensim.utils.simple_preprocess(test_data.iloc[i,0])
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(train_data.iloc[i,0]), [i]) #(gensim.utils.simple_preprocess(line), [i])

In [114]:
train_corpus = list(read_corpus(filepath_train, train_data['cleaned_tweet']))
test_corpus = list(read_corpus(filepath_test, test_data['cleaned_tweet'], tokens_only=True))

In [115]:
train_corpus[:3]

[TaggedDocument(words=['hoe'], tags=[0]),
 TaggedDocument(words=['pass', 'mad', 'hoe', 'give', 'want'], tags=[1]),
 TaggedDocument(words=['thi', 'hoe', 'bad'], tags=[2])]

In [116]:
print(test_corpus[:2])

[['thirsti', 'ass', 'bitch', 'shit', 'dat', 'ion', 'like'], ['wondertrad', 'good', 'pokemon', 'featur', 'ever', 'get', 'level', 'japanes', 'wobufet', 'level', 'begin', 'area', 'trash', 'pokemon']]


In [117]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [118]:
model.build_vocab(train_corpus)

In [119]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 29.3 s, sys: 4.83 s, total: 34.1 s
Wall time: 21.1 s


In [120]:
print(train_data.head())

                                       cleaned_tweet  \
0                                                hoe   
1                             pass mad hoe give want   
2                                        thi hoe bad   
3  eat ya pussi like savag onc put ya nasti ass f...   
4           ur ur friend bitch u hate walk past us n   

                                               tweet  labels  
0               @CassiusClay_232 who all in that hoe       1  
1  Passing up on mad hoes just to give him what h...       1  
2  &#8220;@DreamGirlsOnly: &#128525; http://t.co/...       1  
3  I'll eat ya pussy like a savage, but once you ...       1  
4  RT @khloewithoutheC: @TypicalMinions: when ur ...       1  


In [121]:
# TODO: get vector for every tweet and save to dataframe
model.infer_vector(['pass', 'mad', 'hoe', 'give', 'want'])

array([ 0.18246801, -0.05177258,  0.12900902, -0.40880725,  0.04892077,
        0.04561701,  0.10593386, -0.30332604,  0.5015325 , -0.13664652,
        0.09894111, -0.23027982, -0.02410637,  0.18286698, -0.03510138,
       -0.00163555,  0.08374929, -0.28849176,  0.17920429,  0.14584008,
       -0.04637469,  0.2002474 , -0.22837402,  0.30320618, -0.09123301,
       -0.2578659 ,  0.18679604, -0.08515184, -0.48427072,  0.04025656,
        0.03632876,  0.231319  , -0.12509616,  0.4396587 ,  0.2621449 ,
       -0.10957932, -0.18649799,  0.1458739 ,  0.24972688,  0.24584503,
        0.4771198 , -0.12632135,  0.14044486,  0.04645982, -0.03959664,
        0.056378  ,  0.04697101, -0.08374392, -0.23463666, -0.2609698 ],
      dtype=float32)

In [11]:
# ASSESSING: for later steps if we want to try
'''
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])
    '''

In [12]:
#collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus

Counter({0: 291, 1: 9})

In [13]:
'''
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))
'''

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [14]:
'''
# Pick a random document from the corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))
'''

Train Document (168): «six swiss tour company staff have been found guilty of manslaughter over the deaths of australians and seven others in canyoning disaster two guides who survived the accident on july were acquitted adventure world president stephan friedli vice president peter balmer and director georg hoedle were each given five month suspended sentence and fine of swiss francs about general manager felix oehler received five months and fine of francs base manager bernhard gafner four months and francs and lead guide bernhard steuri three months and francs all six will pay one eighth of the court costs and one eighth of the plaintiffs costs about francs each guides simon wiget and stefan abegglen were acquitted peter dewar whose son bradley died in the disaster says more legal action is planned guess if anything the main thing we were waiting for was verdict of guilty or not guilty mr dewar said the guilty verdict at least leaves something open for civil action we already have l

In [15]:
'''
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))
    '''

Test Document (47): «the european parliament is spoiling for fight with israel it has voted to review the eu diplomatic links with the jewish state to impose an arms embargo and to threaten wider trade sanctions many meps want to go further and dispatch european military force to the region in order to protect the palestinian people»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (76, 0.7424429655075073): «the death toll in argentina food riots has risen to local media reports say four more people died this morning in clashes between police and protesters near the presidential palace in the capital buenos aires president fernando de la rua has called on the opposition to take part in government of national unity and apparently will resign if it does not looting and rioting has generally given way to more peaceful demonstrations against the faltering government blamed for month recession heavily armed police using powers under day state of siege decree a